# CompuOpti

In [1]:
import os
import json

import numpy as np
import gurobipy as grb
from gurobipy import GRB

In [2]:
INF = 2 ** 64

In [3]:
model = grb.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-05


In [4]:
instance = "toy_instance.json"

file = os.path.join("instances", instance)
data = json.load(open(file, "r"))

## Modeling

### Parameters of the problem

Define the length of indices

In [5]:
# Number of workers
worker_length = len(data["staff"])

# Number of jobs
job_length = len(data["jobs"])

# Number of skills
skill_length = len(data["qualifications"])

# Number of days
day_length = data["horizon"]

Define jobs parameters

In [6]:
gains_job = np.array([job["gain"] for job in data["jobs"]])
penalties_job = np.array([job["daily_penalty"] for job in data["jobs"]])
due_dates_job = np.array([job["due_date"] for job in data["jobs"]])
work_days_job_skill = np.array([
    [
        job["working_days_per_qualification"][skill] if skill in job["working_days_per_qualification"] else 0
        for skill in data["qualifications"]
    ]
    for job in data["jobs"]
])

Define staff parameters

In [7]:
qualifications_worker_skill = np.array([
    [
        1 if skill in worker["qualifications"] else 0
        for skill in data["qualifications"]
    ]
    for worker in data["staff"]
])
vacations_worker_day = np.array([
    [
        1 if 1 + day in worker["vacations"] else 0
        for day in range(day_length)
    ]
    for worker in data["staff"]
])

### Decision variables

Variable to model the full solution

In [8]:
# 4-D array of binary variables : 1 if a worker is assigned to a certain project for a certain skill on a certain day, else 0
works_worker_job_skill_day = model.addVars(worker_length, job_length, skill_length, day_length, vtype=GRB.BINARY, name="work")

Variables to compute the total gain

In [9]:
is_realized_job = model.addVars(job_length, vtype=GRB.BINARY, name="is_realized") # 1 if a job is realized, else 0

Variable to compute the maximum duration (and the penalties)

In [10]:
is_worked_job_day = model.addVars(job_length, day_length, vtype=GRB.BINARY, name="is_worked") # 1 if a certain day is a work day for a certain job, else 0
max_duration = model.addVar(vtype=GRB.INTEGER, name="max_duration") # Integer that represents the maximum duration for any job

Variables to compute the maximum assignement

In [11]:
is_assigned_worker_job = model.addVars(worker_length, job_length, vtype=GRB.BINARY, name="is_assigned") # 1 if a certain worker is assigned on a certain job, else 0
max_assigned = model.addVar(vtype=GRB.INTEGER, name="max_assigned") # Integer that represents the maximum number of assigned jobs for any worker

### Constraints

#### Time Table Constraints

Define the constraints of the planning problem itself.

- Worker qualification constraint

In [12]:
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= qualifications_worker_skill[worker, skill]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="qualification",
)

{(0, 0, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 0, 2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1, 0, 2): <gurobi.Constr *Awaiting Model Up

- Uniqueness of the daily assignement : *done with the vacation constraint (see below)*

- Vacation constraint

In [13]:
model.addConstrs(
    (
        grb.quicksum(works_worker_job_skill_day[worker, job, skill, day] for job in range(job_length) for skill in range(skill_length)) \
            <= 1 - vacations_worker_day[worker, day]
        for worker in range(worker_length)
        for day in range(day_length)
    ),
    name="vacation",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>}

- Job coverage constraint

In [14]:
model.addConstrs(
    (
        grb.quicksum(works_worker_job_skill_day[worker, job, skill, day] for worker in range(worker_length) for day in range(day_length)) \
            == is_realized_job[job] * work_days_job_skill[job, skill]
        for job in range(job_length)
        for skill in range(skill_length)
    ),
    name="job_coverage",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>}

- Uniqueness of the realized projects : *done with realized_job either 0 or 1*

#### Variable Constraints

Define the constraints that are related to the definition of additional variables.

- Is realized : *done with job coverage constraint*

- Is worked

In [15]:
# exists_worker_skill works == 1 => is_worked == 1
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= is_worked_job_day[job, day]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="is_worked_job_day",
)
# forall_worker_skill works == 0 => is_worked == 0
model.addConstrs(
    (
        is_worked_job_day[job, day] <= \
            grb.quicksum(
                works_worker_job_skill_day[worker, job, skill, day]
                for worker in range(worker_length)
                for skill in range(skill_length)
            )
        for job in range(job_length)
        for day in range(day_length)
    ),
    name="is_worked_job_day_bis",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,


- Maximum duration

In [16]:
model.addConstrs(
    (
        grb.quicksum(is_worked_job_day[job, day] for day in range(day_length)) \
            <= max_duration
        for job in range(job_length)
    ),
    name="max_duration",
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

- Is assigned

In [17]:
# exists_skill_day works == 1 => is_assigned == 1
model.addConstrs(
    (
        works_worker_job_skill_day[worker, job, skill, day] <= is_assigned_worker_job[worker, job]
        for worker in range(worker_length)
        for job in range(job_length)
        for skill in range(skill_length)
        for day in range(day_length)
    ),
    name="is_assigned_worker_job",
)
# forall_skill_day works == 0 => is_assigned == 0
model.addConstrs(
    (
        is_assigned_worker_job[worker, job] <= \
            grb.quicksum(
                works_worker_job_skill_day[worker, job, skill, day]
                for skill in range(skill_length)
                for day in range(day_length)
            )
        for worker in range(worker_length)
        for job in range(job_length)
    ),
    name="is_assigned_worker_job_bis",
)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>}

- Maximum assigned

In [18]:
model.addConstrs(
    (
        grb.quicksum(is_assigned_worker_job[worker, job] for job in range(job_length)) \
            <= max_assigned
        for worker in range(worker_length)
    ),
    name="max_assigned",
)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>}

### Objectives

In [20]:
# Add primary objective
model.setObjective(
  grb.quicksum(
    gains_job[job] * is_realized_job[job] \
      - penalties_job[job] * grb.quicksum(
        is_worked_job_day[job, day]
        for day in range(due_dates_job[job], day_length)
      )
    for job in range(job_length)
  ),
  GRB.MAXIMIZE,
)

# Add multi-objective functions
model.ModelSense = GRB.MINIMIZE
model.setObjectiveN(
  max_assigned,
  0,
  2,
)
model.setObjectiveN(
  max_duration,
  1,
  1,
)